<a href="https://colab.research.google.com/github/adtitovich/dsml-17/blob/main/m3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Описание задания:
Решить задачу классификации при помощи обучения модели логистической регрессии. Качество модели оценивается путем подсчета метрик TPR, FPR и построения графиков ROC-кривой, Precision-Recall. Данные для обучения модели хранятся в файле athletes.csv, который можно найти в материалах к занятию.

#Этапы работы:
**1. Преобразуйте данные:    
a. проверьте наличие пропущенных значений. Преобразуйте/удалите пропуски по необходимости;    
b. закодируйте категориальные переменные числовыми значениями по необходимости.**

**2. Разделите выборку на обучающее (80% данных) и тестовое (20% данных) подмножества.**

**3. Постройте ROC-кривую с помощью функции sklearn.metrics. roc_curve.**

**4. Вычислите значение ROC-AUC метрики с помощью функции sklearn.metrics.roc_auc_score.**

**5. Реализуйте подсчет метрик TPR, FPR «вручную», без использования готовых функций из библиотеки sklearn.**

**6. Постройте ROC-кривую с помощью вычисленных в п. 5 метрик: объедините графики из п. 3 и п. 6 в один. Сравните, сделайте вывод.**

**7. Постройте график Precision-Recall, используя метрики, посчитанные в п. 5.**

**8. Вычислите значение ROC-AUC метрики, используя метрики, посчитанные в п. 5.**

**9. Сформулируйте выводы по проделанной работе:
a. как по полученным графикам сделать вывод о качестве модели? Как вы оцениваете обученную модель исходя из подсчитанных метрик?
b. *может ли ROC-кривая проходить ниже диагонали?**